# Extracción Datos El mundo Today
### Detección automática de sarcasmos en textos utilizando Transformers
#### Ernesto Francisco Barrionuevo Aragonés 21727751

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import csv
from bs4 import BeautifulSoup
import requests

# Función para obtener la sopa de la página actual
def get_soup(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return soup

# Ruta al controlador de Chrome
PATH = 'chromedriver.exe'

# Configuración del servicio de Chrome
cService = webdriver.ChromeService(executable_path=PATH)

# Inicia el navegador Chrome
driver = webdriver.Chrome(service=cService)

# URL del sitio web a visitar
url = "https://www.elmundotoday.com/"

# Abre la página
driver.get(url)

# Espera unos segundos para que la página cargue completamente
time.sleep(3)

# Hacer clic en el botón 'Consentir' si existe
try:
    button = driver.find_element(By.XPATH, "//button[@class='fc-button fc-cta-consent fc-primary-button']")
    button.click()
    print("Se hizo clic en el botón 'Consentir'.")
except Exception as e:
    print(f"No se encontró el botón 'Consentir' o no se pudo hacer clic: {str(e)}")

time.sleep(3)

# Definir la lista para almacenar los títulos
all_titles = []

# Iterar sobre las páginas y extraer los títulos
for page in range(10000):  
    try:
        # Obtener la sopa de la página actual
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        # Encontrar la sección de los títulos
        section = soup.find('div', {'id': 'tdi_83', 'class': 'td_block_inner'})
        
        # Encontrar los títulos dentro de esa sección
        titles = section.find_all('h3', class_='entry-title td-module-title')

        # Guardar los títulos en la lista all_titles
        for title in titles:
            all_titles.append(title.get_text())

        print(f"Se han extraído {len(titles)} títulos de la página {page + 1}.")
        
        # Hacer clic en el botón 'Next Page'
        try:
            next_button = driver.find_element(By.ID, 'next-page-tdi_83')
            next_button.click()
            print(f"Se hizo clic en el botón 'Next Page' ({page + 1}).")
            time.sleep(2)  # Espera 2 segundos para cargar la nueva página
        except Exception as e:
            print(f"No se pudo hacer clic en el botón 'Next Page' ({page + 1}): {str(e)}")
            break

    except Exception as e:
        print(f"No se pudieron extraer los títulos de la página {page + 1}: {str(e)}")
        break

# Cerrar el navegador al finalizar
driver.quit()

# Guardar los títulos en un archivo CSV
csv_filename = 'Titulos_sarcasmo.csv'
with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Título'])
    for title in all_titles:
        writer.writerow([title])

print(f"Se han guardado los títulos en '{csv_filename}'.")
